## 财新机构评级爬虫

Alex

Created on: 08/07/2021

In [3]:
import os                       # 转换工作路径
import sys                      # 获取当前python文件所在路径
import time                     # 计时

import pandas as pd
import requests                 # 也可以选用selenium，但是requests代码好写易看一点
from bs4 import BeautifulSoup   # 用于格式化网页CSS结构

tic=time.time()         # 计时

url_all ='http://stock.finance.sina.com.cn/stock/go.php/vIR_CustomSearch/index.phtml?&industry=sw2_340400' # 爬取一个行业的全部个股的机构评级
url_root='http://stock.finance.sina.com.cn'

r=requests.get(url_all).text
bs=BeautifulSoup(r)
stock_urls=bs.find_all('a',attrs={'href':True})
stock_urls=[x.get('href') for x in stock_urls]
stock_urls=[x for x in stock_urls if 'vIR_StockSearch/key' in x]
stock_urls

['/stock/go.php/vIR_StockSearch/key/sh600305.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh603027.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh605300.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh605338.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz300783.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz300791.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz002847.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh603517.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh603345.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh603317.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh600872.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh603866.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh603288.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz002597.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz002557.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz300146.phtml',
 '/stock/go.php/vIR_StockSearch/key/sz002507.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh600887.phtml',
 '/stock/go.php/vIR_StockSearch/key/sh600882.p

In [4]:
res_raw=[]
for i,stock_url in enumerate(stock_urls):
    for ii in range(1,30):
        url=url_root+stock_url+'?p=%s'%ii
        rs=requests.get(url)     # 调用爬虫主体对象访问目标url并返回网站text
        r=rs.text           
        bs=BeautifulSoup(r)
        aa=bs.find_all('tr')
        if len(aa)==0:break
        res_raw.extend(aa)
    print((i+1),'/',len(stock_urls),'timer:',round(time.time()-tic,2),'s')

1 / 20 timer: 29.6 s
2 / 20 timer: 56.15 s
3 / 20 timer: 84.96 s
4 / 20 timer: 109.35 s
5 / 20 timer: 136.08 s
6 / 20 timer: 160.05 s
7 / 20 timer: 189.35 s
8 / 20 timer: 218.52 s
9 / 20 timer: 246.78 s
10 / 20 timer: 274.71 s
11 / 20 timer: 304.01 s
12 / 20 timer: 332.3 s
13 / 20 timer: 365.37 s
14 / 20 timer: 393.23 s
15 / 20 timer: 420.76 s
16 / 20 timer: 447.79 s
17 / 20 timer: 476.94 s
18 / 20 timer: 512.64 s
19 / 20 timer: 537.6 s
20 / 20 timer: 563.91 s


In [5]:
df_columns=['股票代码','股票名称','目标价','最新评级', 
            '评级机构', '分析师', '行业', '评级日期']

res=list(map(lambda x: x.get_text().split('\n')[1:9],res_raw))      # 分布式处理每一个comments
res=list(filter(lambda x:x[0]!='股票代码',res))
res=pd.DataFrame(res,columns=df_columns)                            # 将结果存储在dataframe
res.sort_values(['股票代码','评级日期'],inplace=True)
res['量化评级']=res['最新评级'].map({'买入':1,'增持':0.5,'中性':0,'卖出':-1})

res_daily_mean=res[['股票代码','评级日期','量化评级']].groupby(['股票代码','评级日期'],as_index=False)['量化评级'].mean()


In [7]:
res

,股票代码,股票名称,目标价,最新评级,评级机构,分析师,行业,评级日期,量化评级
6547,002507,涪陵榨菜,,买入,西南证券,"朱会振,李光歌",食品加工,2018-11-19,1.0
6546,002507,涪陵榨菜,,增持,招商证券,"杨勇胜,于佳琦,欧阳予,李泽明",食品加工,2018-11-26,0.5
6544,002507,涪陵榨菜,,买入,东北证券,"李强,马雪薇",食品加工,2018-11-29,1.0
6545,002507,涪陵榨菜,,增持,兴业证券,"陈娇,颜言",食品加工,2018-11-29,0.5
6543,002507,涪陵榨菜,,买入,申万宏源研究,"吕昌,周缘",食品加工,2018-12-05,1.0
...,...,...,...,...,...,...,...,...,...
845,605338,巴比食品,42.81,增持,国泰君安,"訾猛,李梓语",食品加工,2021-07-30,0.5
837,605338,巴比食品,,增持,中泰证券,"范劲松,晏诗雨,熊欣慰",食品加工,2021-08-01,0.5
838,605338,巴比食品,,买入,天风证券,"刘章明,刘畅,李珍妮",食品加工,2021-08-01,1.0
839,605338,巴比食品,,买入,华安证券,文献,食品加工,2021-08-01,1.0
